# Best model

Redefinimos la funcion de preproceso, usada en cuadernos anteriores, quitando el randomundersampler para no perder ningún usuario por el camino. La función final será más escueta, pues solo necesitamos la última parte de esta.

In [4]:
def preproceso(df, mas_comunes = None):
    import pandas as pd
    from sklearn.preprocessing import StandardScaler
    from imblearn.under_sampling import RandomUnderSampler

    # Cargar etiquetas y unirlas al dataframe original
    df_labels = pd.read_csv("train_labels.csv")
    df = pd.merge(df, df_labels, how="left", on="ID")

    # Eliminar columnas con más del 90% de datos faltantes
    pct_missing_columns = df.isnull().mean() * 100
    cols_to_drop = pct_missing_columns[pct_missing_columns >= 90].index.to_list()
    df.drop(cols_to_drop, axis=1, inplace=True)

    # Eliminar filas con más del 75% de datos faltantes
    pct_missing_rows = df.isnull().mean(axis=1) * 100
    rows_to_drop = pct_missing_rows[pct_missing_rows >= 75].index.to_list()
    df.drop(rows_to_drop, axis=1, inplace=True)

    # Imputar los valores faltantes con la moda de cada columna
    df = df.apply(lambda col: col.fillna(col.mode()[0]))


    # Procesar la columna de fechas
    df["Expenditure_AHF"] = pd.to_datetime(df["Expenditure_AHF"])
    df["year"] = df["Expenditure_AHF"].dt.year
    df["month"] = df["Expenditure_AHF"].dt.month
    df["day"] = df["Expenditure_AHF"].dt.day
    df.drop("Expenditure_AHF", axis=1, inplace=True)

    # Convertir la columna 'ID' en una variable dummy y conservarla separada
    ID = df.ID
    df = pd.get_dummies(df.drop("ID", axis=1))
    df["ID"] = ID

    # Eliminar columnas con más del 80% de valores iguales a 1 o 0
    only_ones = (df == 1).mean() * 100
    only_ones = only_ones[only_ones >= 80].index.to_list()
    only_zeros = (df == 0).mean() * 100
    only_zeros = only_zeros[only_zeros >= 80].index.to_list()
    borrar = only_ones + only_zeros
    df.drop(borrar, axis=1, inplace=True)

    # Normalizar las columnas numéricas
    numerales = df.select_dtypes(include="number").columns.to_list()
    scaler = StandardScaler()
    df[numerales] = scaler.fit_transform(df[numerales])

    df = df[mas_comunes]
    return df

In [5]:
def final(ruta_archivo, cols, tamaño_fragmento=100000):
    import pandas as pd
    dfs_list=[]
    for i, fragmento in enumerate(pd.read_csv(ruta_archivo, chunksize=tamaño_fragmento)):
        dfs_list.append(preproceso(fragmento, cols))
        print(i)

    dfs_final = pd.concat(dfs_list, ignore_index = True)
    grouped_df = dfs_final.groupby("ID").agg(["mean", "std"])
    grouped_df.columns = [f"{col}_{stat}" for col, stat in grouped_df.columns]
    grouped_df.fillna(0, inplace=True)
    return grouped_df

Traemos las columnas de forma directa desde processing y le aplicamos la función final para que los datos estén a la par con los que entrenaron el modelo

In [7]:
cols = ['Payment_6804', 'Infraction_CGP', 'Base_7744', 'Base_80863', 'Risk_1930', 'Expenditure_JIG', 'Infraction_SNZ', 'Base_02683', 'Infraction_SBF',
        'Infraction_ZWWJ', 'Infraction_QJJF', 'Base_76065', 'Infraction_EJZ', 'Base_6872', 'Risk_0322', 'Infraction_FMXQ', 'Infraction_GGO', 'ID']


test_data = final("test_data.csv", cols)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


Cargamos el mejor modelo entrenado en supervised (RandomForest) y predecimos los labels de test_data

In [9]:
import pickle
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Cargamos el modelo desde el archivo
with open('modelo_randomforest.pkl', 'rb') as file:
    modelo_cargado = pickle.load(file)

# Hacemos predicciones en el conjunto de prueba
test_labels = modelo_cargado.predict(test_data)

In [10]:
len(test_labels)

183566

Exportamos los datos para su evaluación, no sin antes ponerlos en un dataframe o serie para poder usar un método

In [12]:
import pandas as pd
test_labels = pd.DataFrame(test_labels, columns = ["labels"])

In [13]:
test_labels

,labels
0,1
1,1
2,0
3,1
4,0
...,...
183561,0
183562,0
183563,0
183564,0


In [14]:
test_labels.to_csv("test_labels.csv", index=False)